In [ ]:
import time
from scraper_WoS import WosClient
import pandas as pd


In [ ]:
soap = WosClient(lite=True)
soap.connect()

In [ ]:
#getting all the names 
searchnames = []
i=0
for line in open('../Data/Dismiss_Acad_List.txt','r'):
    QueryString = line.strip()
    QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
    print('QueryString:', QueryString)
    searchnames.append(QueryString)


In [ ]:
len(searchnames)

In [ ]:
len(searchnames)
pd.DataFrame(searchnames,columns=['SearchNames']).to_csv('SearchNames.csv')
#save Search names in csv file
SearchNames = pd.read_csv('SearchNames.csv')
SearchNames.SearchNames[0].strip().split('=')[1]

In [ ]:

soap = WosClient(lite=True)
soap.connect()
#query = "VA= Article"
res=soap.search(QueryString)


In [ ]:
names = []
lens = []
co_aut = []
i=0
for i in range(len(SearchNames)):
    line = SearchNames.SearchNames[i]
    QueryString = line
    print(QueryString)
    if i == 2400: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
        soap = WosClient(lite=True)
        soap.connect()
    results = soap.search(QueryString)
    #getting number of name of co-authors
    if (results.recordsFound>0):
        authors = []
        for j in range (len(results.records)):
            for k in range (len(results.records[j].authors)):
                for l in range (len(results.records[j].authors[k].value)):
                    co_authors=results.records[j].authors[k].value[l]
                    authors.append(co_authors)
                    print('co_author',co_authors)
                #co_aut.append(authors)
            #co.append(co_aut)
    print(results.recordsFound)
    names.append(QueryString.split('=')[1])
    lens.append(results.recordsFound)
    co_aut.append(authors)
    time.sleep(0.5) # WOS constraints that you can only search twice per second
    i=i+1

In [ ]:
#a python module to make unicode strings work as expected for turkish chars. solves the turkish "İ" problem

# have to do pip install unicode_tr
names_en = []
from unicode_tr.extras import slugify
for i in range(len(names)):
    name_en = slugify(names[i])
    name_en = name_en.replace('-',' ').title()
    names_en.append(name_en)

In [ ]:
len(names)

In [ ]:
df = pd.DataFrame([name,lens,co_aut]).T
df.columns=['Author','NoPapers','CoAuthors']
df

In [ ]:
#formatting co authors column to match with authors
#for i in range(df.shape[0]):
#    for j in range(len(df.CoAuthors.iloc[i])):
#        a=df.CoAuthors.iloc[i]
#        a[j]=a[j].replace(","," ")
#        df.CoAuthors.iloc[i] = a
#df

In [ ]:
len(names)

In [ ]:
cnt=0
for l in co_aut:
    cnt += len(l)
print('Totally ' + str(cnt) + ' guys were downloaded as coauthors')

In [ ]:
coauthors = set()
for l in co_aut:
    for author in l:
        author = author.replace(',', '')
        coauthors.add(author)

In [ ]:
# let's now create the file with not-dismissed professors:
dismissed = set(names_en)

print('Totally we have ' + str(len(dismissed)) + ' dismissed and ' + str(len(coauthors)) + ' unique coauthors')

In [ ]:
undismissed = coauthors - dismissed
print('So, we have ' + str(len(undismissed)) + ' undismissed guys')

In [ ]:
dismissed_df = pd.DataFrame()
dismissed_df['AU']=list(dismissed)
dismissed_df.to_csv('../data/dismissed.csv')
undismissed_df = pd.DataFrame()
undismissed_df['AU']=list(undismissed)
undismissed_df.to_csv('../data/undismissed.csv')